# Forward Accumulation

The forward accumulation method is method that performs automatic differentiation through the use of a *computational graph*. Computational graphs are useful because can represent functions in a visual way where the nodes are the operations apply to an input or constant, and the nodes are the inputs.

Forward accumulation passing through the computational graph only once, it's like apply the chain rule iteratively in each operation. When perform an operation, in fact we are evaluating the function. In order to use the chain rule in each operation we also need to compute the derivative in that point, by using **dual numbers** the algorithm can compute the function and derivative in the same operation.

A *dual number* is expressed as $a + b\epsilon$, where $a, b \in \mathbb{R}$, and $\epsilon$ is an abstract quantity that has the property of $\epsilon^2 = 0$. The sum and multiplication between two dual numbers is defines as follows:

$$
(a + b\epsilon) + (c + d\epsilon) = (a + c) + (b + d)\epsilon
$$
$$
(a + b\epsilon) \times (c + d\epsilon) = ac + ad\epsilon  + bc\epsilon + bd\epsilon^2 = (ac) + (ad + bc)\epsilon
$$

An interesting property of dual numbers is that we can get the evaluation and derivative $f(a)$ and $f'(a)$ respectively of a smooth function $f$  if we evaluate $f$ at $a + b\epsilon$, this can be proved using the Taylor series:

$$f(x) = \sum_{n = 0}^{\infty} \frac{f^{(n)}(a)}{n!}(x-a)^n$$

If we define $x = a + b\epsilon$:

$$
f(x) = f(a + b\epsilon) = \sum_{n = 0}^{\infty} \frac{f^{(n)}(a)}{n!}( a + b\epsilon - a)^n
$$
$$
f(a + b\epsilon) = \sum_{n = 0}^{\infty} \frac{f^{(n)}(a)}{n!}(b\epsilon)^n
$$
$$
f(a + b\epsilon) = f(a) + bf'(a)\epsilon + \sum_{n = 2}^{\infty} \frac{f^{(n)}(a)}{n!}(b\epsilon)^n
$$
Given that $\epsilon^2 = 0$
$$
\sum_{n = 2}^{\infty} \frac{f^{(n)}(a)}{n!}(b\epsilon)^n = \epsilon^2 \sum_{n = 2}^{\infty} \frac{f^{(n)}(a)}{n!} b^n \epsilon^{n-2} = 0
$$

We obtain the expression shown below:
$$
f(a + b\epsilon) = f(a) + bf'(a)\epsilon
$$

If $b = 1$, then:

$$f(a + \epsilon) = f(a) + f'(a)\epsilon$$

## Implementing dual numbers in Julia

In **Julia** we can use the keyword `struct` to define a structure and perform operations with Dual numbers:

In [1]:
struct Dual
    α::Float64
    β::Float64
end

By using `Base.:`, **Julia** allow us to overload some operators:

In [2]:
# Define the '+' operator for dual numbers
Base.:+(x::Dual, y::Dual) = Dual(x.α + y.α, x.β + y.β)
# Define the '*' operator for dual numbers
Base.:*(x::Dual, y::Dual) = Dual(x.α*y.α, x.α*y.β + x.β*y.α)
# Define the '*' operator for multiplication between float and  Dual number
Base.:*(a::Float64, x::Dual) = Dual(a*x.α, a*x.β)

## Mathematical functions with dual numbers

It is necessary define the mathematical functions for dual numbers, some popular functions are implemented below using the fact that $f(a + b\epsilon) = f(a) + bf'(a)\epsilon$.

### $\ln{x}$

For natural logarithmic $f(x) = \ln{x}$ and $f'(x) = \frac{1}{x}$:

$$
ln(a + b\epsilon) = ln(a) + \frac{b}{a}\epsilon
$$

### $\cos{x}$

For cosine  $f(x) = \cos{x}$ and $f'(x) = -\sin{x}$:

$$
\cos{(a + b\epsilon)} = \cos{a} - b\epsilon\sin{a}
$$

### $\sin{x}$

For sine  $f(x) = \sin{x}$ and $f'(x) = \cos{x}$:

$$
\sin{(a + b\epsilon)} = \sin{a} + b\epsilon\cos{a}
$$

### $x^n$

Here $f(x) = x^n$ and $f'(x) = nx^{n-1}$:

$$
(a + b\epsilon)^n = a^n + bn(a)^{n-1}\epsilon
$$


In [3]:
# Define log() for Dual numbers
Base.log(x::Dual) = Dual(log(x.α), x.β/x.α)
# Define cos() for Dual numbers
Base.cos(x::Dual) = Dual(cos(x.α), -x.β*sin(x.α))
# Define sin() for Dual numbers
Base.sin(x::Dual) = Dual(sin(x.α), x.β*cos(x.α))
# Define power(x, n) for Dual numbers
power(x::Dual, n) = n != 0 ? Dual(x.α^n, x.β*n*(x.α^(n-1))) : Dual(1.0, 0.0)

power (generic function with 1 method)

Let's evaluate the function and compute the derivative:

For $f(x) = 2x^3$ at $x=2$:

$$
f(2) = 16
$$
$$
f'(x) = 6x^2 \, , \,
f'(2) = 24
$$

Defining $x = 2 + \epsilon$:

In [4]:
x = 2

2x^3, 6x^2

(16, 24)

In [5]:
x = Dual(2.0, 1.0)

2.0power(x, 3.0)

Dual(16.0, 24.0)

For $f(x) = \sin{3x}$ at $x=\pi/4$:

$$
f(\pi/4) = 0.707106
$$
$$
f'(x) = 3\cos{3x} \, , \,
f'(\pi/4) = -2.12132
$$

Defining $x = \pi/4 + \epsilon$:

In [6]:
x = π/4

sin(3x), 3cos(3x)

(0.7071067811865476, -2.1213203435596424)

In [7]:
x = Dual(π/4, 1)

sin(3.0x)

Dual(0.7071067811865476, -2.1213203435596424)

Now let's draw a computational graph and use forward accumulation to compute $\partial f / \partial y$ at $(x, y) = (\pi/6, \pi/3)$, where $f(x, y) = \sin{(x + y^3)}$:

![Computational graph](Images/ComputationalGraph.png)



In [8]:
X = Dual(π/6, 0.0)
Y = Dual(π/3, 1.0)

sin(X + power(Y, 3.0))

Dual(0.994885359398551, -0.33231123398036866)

These values are equal that those which are obtained by computing directly $\frac{\partial f}{\partial y}$:

$$
\frac{\partial f}{\partial y} = 3y^2cos(x + y^3)
$$

$$
\frac{\partial f}{\partial y}(\pi/6, \pi/3) = -0.332311
$$

In [9]:
3*(π/3)^2*cos(π/6 + (π/3)^3)

-0.33231123398036866